In [ ]:
from flask import Flask, render_template, request, jsonify, send_from_directory
from flask import Flask, request, send_file, abort, Response
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

app = Flask(__name__)

# Set up directories for uploads and processed videos
UPLOAD_FOLDER = 'static/uploads'
PROCESSED_FOLDER = 'static/processed'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

# Load the fine-tuned MoviNet model
saved_model_path="C:/Users/amazm/University/Final Year Project/Website/saved_model"
yolo_model=YOLO('yolov8n.pt')

def preprocess_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to model input size
    frame = frame / 255.0  # Normalize to range [0, 1]
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    frame = np.expand_dims(frame, axis=0)  # Add temporal dimension
    return frame.astype(np.float32)

def detect_objects(frame):
    """Detect persons in a frame using YOLOv8."""
    results = yolo_model(frame)
    detections = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes
    confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)  # Class IDs
    return [(bbox, conf) for bbox, conf, cls_id in zip(detections, confidences, class_ids) if cls_id == 0]  # Filter persons



def process_video(input_video_path, output_video_path):
    print(f"Input_video={input_video_path}")
    model = tf.saved_model.load(saved_model_path)
    infer = model.signatures["serving_default"]
    #yolo_model=YOLO('yolov8n.pt')
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    print("video open")
    if not cap.isOpened():
        print("Error: Could not open the input video.")
        exit()

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print('frame,height,width got')
# Define codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    print('out declared')
# Variables to track fall detection and bounding box
    fall_detected = False
    tracker = None  # To hold the OpenCV tracker
    fall_occurred = False  # To track if fall has already occurred
    print('variable initialized')
# Process video frame by frame
    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

    # Initialize tracker if not already initialized
        if tracker is None and not fall_occurred:
            # Detect persons using YOLOv8
            detections = detect_objects(frame)
            print('frame detected')
            if detections:
                bbox, _ = detections[0]  # Take the first detected person
                x1, y1, x2, y2 = map(int, bbox)
                tracker = cv2.TrackerCSRT_create()  # Robust tracker
                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))  # Initialize tracker with bounding box

    # Update tracker
        if tracker is not None:
            success, tracked_bbox = tracker.update(frame)
            if success:
                x, y, w, h = map(int, tracked_bbox)
            else:
                tracker = None  # Reset tracker if tracking fails

    # Preprocess the frame for MoviNet
        processed_frame = preprocess_frame(frame)
        print('frame preprocessed')
        try:
        # Run inference
            predictions = infer(image=processed_frame)
            raw_output = predictions['classifier_head_2'].numpy()
            fall_probability = raw_output[0][0]  # Probability for "fall" class
            nofall_probability = raw_output[0][1]  # Probability for "nofall" class
            fall_detected = fall_probability > nofall_probability
        except Exception as e:
            print(f"Error during MoviNet inference: {e}")

    # Once fall is detected, keep the rectangle red and text "FALL DETECTED"
        if fall_detected and not fall_occurred:
            fall_occurred = True  # Mark that fall has occurred

    # Draw bounding box and text
        if tracker is not None:
            color = (0, 0, 255) if fall_occurred else (0, 255, 0)  # Red for fall, green otherwise
            thickness = 5  # Thicker bounding box for better visibility
            label = "FALL DETECTED" if fall_occurred else "Person"
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, thickness)
        # More elegant text: bigger and shadowed
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
        print('Bounding box')
    # Overlay status text with improved design
        status_text = "FALL DETECTED" if fall_occurred else "NO FALL"
        status_color = (0, 0, 255) if fall_occurred else (0, 255, 0)
    # Add shadow to status text for better visibility
        cv2.putText(frame, status_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 5, cv2.LINE_AA)
        cv2.putText(frame, status_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, status_color, 3, cv2.LINE_AA)
        print('Overlay')
    # Adjust bounding box around the person if necessary (in case the person is outside the box)
        if fall_occurred:
        # Detect the person again if fall occurred and update tracker accordingly
            detections = detect_objects(frame)
            if detections:
                bbox, _ = detections[0]  # Get updated bounding box
                x1, y1, x2, y2 = map(int, bbox)
            # Increase bounding box size to ensure it fully includes the person after falling
                margin = 20  # Add margin to the bounding box
                x1, y1 = max(0, x1 - margin), max(0, y1 - margin)
                x2, y2 = min(width, x2 + margin), min(height, y2 + margin)
                x, y, w, h = x1, y1, x2 - x1, y2 - y1
                tracker = cv2.TrackerCSRT_create()  # Reinitialize the tracker to better fit the new position
                tracker.init(frame, (x, y, w, h))

    # Write the frame to the output video
        out.write(frame)
        print('frame saved')
        frame_number += 1

# Release resources
    cap.release()
    out.release()

# Report results
    print(f"Video saved to: {output_video_path}")
# Flask Routes
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/FallDetection')
def FallDetection():
    return render_template('FallDetection.html')

@app.route('/live-detection')
def livedetection():
    return render_template('live-detection.html')

@app.route('/Login')
def Login():
    return render_template('Login.html')

@app.route('/Mental')
def Mental():
    return render_template('Mental.html')

@app.route('/ModulesPage')
def ModulesPage():
    return render_template('ModulesPage.html')

@app.route('/Report')
def Report():
    return render_template('Report.html')

@app.route('/SignUp')
def SignUp():
    return render_template('SignUp.html')
    
@app.route('/upload-video', methods=['GET', 'POST'])
def uploadvideo():
    if request.method == 'POST':
        if 'video' not in request.files:
            return jsonify({'error': 'No video file found in the request'}), 400
        
        video = request.files['video']
        if not video.filename.endswith(('.mp4', '.avi', '.mov', '.webm')):
            return jsonify({'error': 'Unsupported video format'}), 400
        
        upload_path = os.path.join(UPLOAD_FOLDER, video.filename)
        upload_path = os.path.normpath(upload_path).replace("\\", "/")
        print(f"Video uploaded to: {upload_path}")
        video.save(upload_path)
        
        file_name, file_extension = os.path.splitext(video.filename)
        new_extension = '.mp4' if file_extension.lower() != '.avi' else '.avi'

        processed_path = os.path.join(PROCESSED_FOLDER, f"{file_name}{new_extension}")
        processed_path = os.path.normpath(processed_path).replace("\\", "/")
        print(f"Processing video: {processed_path}")

        try:
            process_video(upload_path, processed_path)  # Process the video
        except Exception as e:
            print(f"Error processing video: {e}")
            return jsonify({'error': f"Processing failed: {str(e)}"}), 500

        if not os.path.exists(processed_path):
            return jsonify({'error': 'Processed video not found'}), 500

        processed_video_url = f'/static/processed/{os.path.basename(processed_path)}'
        print(f"Processed video URL: {processed_video_url}")
        return jsonify({'processedVideoUrl': processed_video_url})
    
    return render_template('upload-video.html')

    
@app.route('/download/<filename>')
def download_processed_video(filename):
    """ Serve processed video as a file download """
    file_path = os.path.join(PROCESSED_FOLDER, filename)

    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found.")
        return abort(404)

    try:
        return send_file(file_path, as_attachment=True)
    except Exception as e:
        print(f"Error sending file: {e}")
        return jsonify({'error': f"Download failed: {str(e)}"}), 500
        
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jan/2025 20:08:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2025 20:08:56] "GET /static/doc.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:56] "GET /static/detect.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:56] "GET /static/report.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:56] "GET /static/mental.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:58] "GET /ModulesPage HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2025 20:08:58] "GET /static/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:58] "GET /static/detect.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:58] "GET /static/report.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:08:58] "GET /static/mental.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:09:01] "GET /FallDetection HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2025 20:09:01] "GET /static/fall-2.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [24/Jan/2025 20:09:01] "GET /static/

Video uploaded to: static/uploads/video.mp4
Processing video: static/processed/video.mp4
Input_video=static/uploads/video.mp4
video open
frame,height,width got
out declared
variable initialized

0: 480x640 1 person, 108.6ms
Speed: 0.0ms preprocess, 108.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
frame detected
frame preprocessed
Bounding box
Overlay

0: 480x640 1 person, 47.7ms
Speed: 12.0ms preprocess, 47.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
frame saved
frame preprocessed
Bounding box
Overlay

0: 480x640 1 person, 45.9ms
Speed: 15.5ms preprocess, 45.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
frame saved
frame preprocessed
Bounding box
Overlay

0: 480x640 1 person, 48.0ms
Speed: 9.9ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
frame saved
frame preprocessed
Bounding box
Overlay

0: 480x640 1 person, 49.9ms
Speed: 2.1ms preprocess, 49.9ms inference, 1.0ms postproce